In [66]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb

In this task you will have to solve a standard problem on tabular data. However, the final solution must be obtained using the function **clf_train**.

## Metric

squared [RMSE](https://en.wikipedia.org/wiki/Root_mean_square_deviation) 
$$ SCORE = (\sum_{i=1}^{n}{(true_i - predict_i)^{2}}/n)^{1/4} $$
* **true** - real value of target
* **predict** - your predict
* **n** - length of target

## Restriction

You cannot change the code of the **clf_train** function. You can only use submissions produced by this function. This function takes the following s input: training and test datasets with the features preprocessed by you, weigh
ts of target for training, id column for generating the sample_submission.csv and function for inverting the target. 

## Data

* **train_tables_Estudiantes.csv** - train dataset with 9 numeric features, 3 datetime features and target
* **train_tables_Validar.csv** - test dataset with 9 numeric features, 3 datetime features and id for submission
* **sample_submission.csv** -  example of submission file with id coluns and target column that needed to predict.

read train and test dataframes

In [67]:
df = pd.read_csv('train_tables_Estudiante.csv')
df.isna().sum()

feat_0    2704
feat_1    2660
feat_2    3364
feat_3    3465
feat_4    2708
feat_5    3488
feat_6    3335
feat_7    3545
feat_8    3280
day       3522
hour      3174
minute    2638
target       0
dtype: int64

In [68]:
df.head()
len(df)

30704

In [69]:
n = int(0.8 * len(df))

train = df.iloc[0:n,:]
print(train.info())
len(train)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24563 entries, 0 to 24562
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   feat_0  22384 non-null  float64
 1   feat_1  22447 non-null  float64
 2   feat_2  21889 non-null  float64
 3   feat_3  21782 non-null  float64
 4   feat_4  22402 non-null  float64
 5   feat_5  21787 non-null  float64
 6   feat_6  21897 non-null  float64
 7   feat_7  21720 non-null  float64
 8   feat_8  21933 non-null  float64
 9   day     21787 non-null  float64
 10  hour    22016 non-null  float64
 11  minute  22444 non-null  float64
 12  target  24563 non-null  float64
dtypes: float64(13)
memory usage: 2.4 MB
None


24563

In [70]:
print(len(df))

test = df.iloc[n:len(df),:]
test.head()

30704


,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,day,hour,minute,target
24563,441.678002,NaN,20.499042,83.582179,2860.389685,48.634757,2421.411334,1322.335612,647.417832,1.0,12.0,8.0,677.048729
24564,445.430290,2573.193068,20.761869,84.700180,2849.054200,49.395417,2502.915317,1312.769687,646.773078,1.0,15.0,51.0,703.265564
24565,NaN,2583.480172,20.182630,84.106913,2889.151897,49.316543,2517.708428,1317.302558,650.437829,1.0,13.0,55.0,708.205771
24566,472.551513,2773.495373,NaN,86.160914,2859.806499,51.013330,2730.414193,1327.419555,641.775582,NaN,4.0,13.0,770.099617
24567,519.573717,3018.812626,20.487228,88.225287,2828.256797,53.430065,2925.473706,1358.437424,NaN,1.0,4.0,4.0,842.973039


In [71]:
# Imputar valores faltantes en numéricas con la mediana
num_cols = [c for c in train.columns 
            if train[c].dtype in [np.float64, np.int64] 
            and c != 'target']

In [72]:
for col in num_cols:
    med = train[col].median()
    # reasignamos el fillna en lugar de usar inplace
    train.loc[:, col] = train[col].fillna(med)
    test.loc[:, col] = test[col].fillna(med)

In [77]:
test['id'] = range(len(test))
test

C:\Users\Estudiante\AppData\Local\Temp\ipykernel_19268\4036794867.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['id'] = range(len(test))


,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,day,hour,minute,target,id
24563,441.678002,2563.415643,20.499042,83.582179,2860.389685,48.634757,2421.411334,1322.335612,647.417832,1.0,12.0,8.0,677.048729,0
24564,445.430290,2573.193068,20.761869,84.700180,2849.054200,49.395417,2502.915317,1312.769687,646.773078,1.0,15.0,51.0,703.265564,1
24565,444.257633,2583.480172,20.182630,84.106913,2889.151897,49.316543,2517.708428,1317.302558,650.437829,1.0,13.0,55.0,708.205771,2
24566,472.551513,2773.495373,20.349846,86.160914,2859.806499,51.013330,2730.414193,1327.419555,641.775582,1.0,4.0,13.0,770.099617,3
24567,519.573717,3018.812626,20.487228,88.225287,2828.256797,53.430065,2925.473706,1358.437424,646.456612,1.0,4.0,4.0,842.973039,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30699,326.482249,1710.906684,20.250298,76.688665,2953.542202,41.612632,1651.832479,1208.388522,646.140649,2.0,9.0,46.0,402.556999,6136
30700,444.792384,2443.515733,20.426957,82.517395,2878.913142,49.831491,2519.665064,1298.910457,643.656504,2.0,10.0,6.0,663.274880,6137
30701,454.793522,2673.987654,20.511876,84.770555,2840.775292,50.483601,2598.763207,1329.824405,646.456612,1.0,16.0,28.0,731.554126,6138
30702,442.780592,2549.803240,20.135048,83.397591,2858.753972,49.699931,2481.478798,1315.159261,650.456487,2.0,2.0,14.0,701.208103,6139


Inference function

**You cannot change this function.**

In [78]:
def clf_train(train, test, target, weight_col, id_col, name_file = 'sub.csv', func_inv = None):

    param = {
    'learning_rate': 0.1,
    'num_leaves': 48,
    'lambda_l1' : 1,
    'lambda_l2' : 1,
    'min_data_in_leaf' : 100,
    'objective': 'mae',
    'verbosity':-1,
    }
    
    predict_test = np.zeros(len(test))

    tr = lgb.Dataset(train, target, weight=weight_col)
    bst = lgb.train(param, tr, num_boost_round=500)
    predict_test = bst.predict(test)
    if func_inv:
        predict_test = func_inv(predict_test)
    sub = pd.DataFrame()
    sub['id'] = id_col
    sub['target'] = predict_test
    sub.to_csv(name_file, index = None)

Function to change target if you need it

In [79]:
def func_inv(x):
    return x 

Train and inference. 
You should use **clf_train** for generating submission

In [80]:
drop_cols = ['target']
train_cols = [c for c in train.columns if c not in drop_cols]
weight = np.ones(len(train))
test_sub = clf_train(train[train_cols], test[train_cols], train['target'] , weight, test['id'].tolist(), 'submission.csv', func_inv = func_inv)

In [81]:
# Generar submission.csv
drop_cols  = ['target']
train_cols = [c for c in train.columns if c not in drop_cols]
weight     = np.ones(len(train))

clf_train(
    train[train_cols],
    test [train_cols],
    train['target'],
    weight,
    test['id'].tolist(),
    name_file='submission.csv',
    func_inv=func_inv
)

print("✔️ Generado submission.csv con columnas ['id','target'].")

# 8) (Opcional) Visualiza las primeras filas
sub = pd.read_csv('submission.csv')
print(sub.head())

✔️ Generado submission.csv con columnas ['id','target'].
   id      target
0   0  675.550311
1   1  705.120000
2   2  705.400283
3   3  767.314400
4   4  845.658323
